In [5]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# Load and clean data
df = pd.read_csv('data/Melbourne_housing_FULL.csv')
df = df[['Price', 'Rooms', 'Bathroom', 'Landsize', 'Suburb', 'Date',
         'BuildingArea', 'Bedroom2', 'Car', 'YearBuilt']].dropna()
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['Month'] = df['Date'].dt.to_period('M').astype(int)

# Log-transform the price
df['log_price'] = np.log(df['Price'])


# Helper function to filter valid test rows
def filter_valid_test(df_test, df_train):
    valid_suburbs = df_train['Suburb'].unique()
    valid_months = df_train['Month'].unique()
    return df_test[df_test['Suburb'].isin(valid_suburbs) & df_test['Month'].isin(valid_months)].copy()

# Define the evaluate_model function
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

# Define formulas using log_price
base_formula = 'log_price ~ BuildingArea + Bedroom2 + Car + YearBuilt + Rooms + Bathroom + Landsize + C(Suburb) + C(Month)'
extended_formula = base_formula + ' + Cluster + DistanceToCentroid'

# Define features for scaling
features_to_scale = ['BuildingArea', 'Bedroom2', 'Car', 'YearBuilt', 'Rooms', 'Bathroom', 'Landsize', 'Month']


cluster_range = range(2, 16, 2)
results_list = []

for k in cluster_range:
    # Scale features before clustering
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df[features_to_scale]) # Scale selected features

    # KMeans clustering
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    df['Cluster'] = kmeans.fit_predict(scaled)
    centroids = kmeans.cluster_centers_
    df['DistanceToCentroid'] = np.linalg.norm(scaled - centroids[df['Cluster']], axis=1)

    # Split data
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Filter test sets
    test_base = filter_valid_test(test_df, train_df).copy()
    test_ext = filter_valid_test(test_df, train_df).copy()

    # Fit OLS models
    base_model = smf.ols(base_formula, data=train_df).fit()
    extended_model = smf.ols(extended_formula, data=train_df).fit()

    test_base['PredictedLog_Base'] = base_model.predict(test_base)
    test_ext['PredictedLog_Ext'] = extended_model.predict(test_ext)

    # Evaluate OLS
    rmse_base, mae_base, r2_base = evaluate_model(test_base['log_price'], test_base['PredictedLog_Base'])
    rmse_ext, mae_ext, r2_ext = evaluate_model(test_ext['log_price'], test_ext['PredictedLog_Ext'])


    features_rf_base = ['BuildingArea', 'Bedroom2', 'Car', 'YearBuilt', 'Rooms', 'Bathroom',
                    'Landsize', 'Month']
    features_rf_extended = ['BuildingArea', 'Bedroom2', 'Car', 'YearBuilt', 'Rooms', 'Bathroom',
                        'Landsize', 'Cluster', 'DistanceToCentroid', 'Month']
    target_rf = 'log_price'

    # Random Forest features
    X_base = df[features_rf_base]
    X_ext = df[features_rf_extended]
    y = df[target_rf]

    X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X_base, y, test_size=0.2, random_state=42)
    X_train_ext, X_test_ext, y_train_ext, y_test_ext = train_test_split(X_ext, y, test_size=0.2, random_state=42)

    rf_base = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rf_base.fit(X_train_base, y_train_base)
    y_pred_base = rf_base.predict(X_test_base)
    rmse_rf_base = np.sqrt(mean_squared_error(y_test_base, y_pred_base))
    mae_rf_base = mean_absolute_error(y_test_base, y_pred_base)
    r2_rf_base = r2_score(y_test_base, y_pred_base)

    rf_ext = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    rf_ext.fit(X_train_ext, y_train_ext)
    y_pred_ext = rf_ext.predict(X_test_ext)
    rmse_rf_ext = np.sqrt(mean_squared_error(y_test_ext, y_pred_ext))
    mae_rf_ext = mean_absolute_error(y_test_ext, y_pred_ext)
    r2_rf_ext = r2_score(y_test_ext, y_pred_ext)

    results_list.append({
        "Clusters": k,
        "OLS Base R²": base_model.rsquared,
        "OLS Base Test R²": r2_base,
        "OLS Ext Test R²": r2_ext,
        "OLS Ext Train R²": extended_model.rsquared,
        "OLS Ext RMSE": rmse_ext,
        "OLS Ext MAE": mae_ext,
        "RF Base R²": rf_base.score(X_train_base, y_train_base),
        "RF Base Test R²": r2_rf_base,
        "RF Ext R²": rf_ext.score(X_train_ext, y_train_ext),
        "RF Ext Test R²": r2_rf_ext,
        "RF Ext RMSE": rmse_rf_ext,
        "RF Ext MAE": mae_rf_ext
    })

# Convert results to DataFrame and display
results_cluster_df = pd.DataFrame(results_list)
print(results_cluster_df.round(3))
results_cluster_df.to_csv('results/results_cluster.csv', index=False)

   Clusters  OLS Base R²  OLS Base Test R²  OLS Ext Test R²  OLS Ext Train R²  \
0         2        0.772             0.745            0.774             0.792   
1         4        0.772             0.745            0.782             0.797   
2         6        0.772             0.745            0.765             0.785   
3         8        0.772             0.745            0.784             0.800   
4        10        0.772             0.745            0.766             0.786   
5        12        0.772             0.745            0.774             0.793   
6        14        0.772             0.745            0.767             0.785   

   OLS Ext RMSE  OLS Ext MAE  RF Base R²  RF Base Test R²  RF Ext R²  \
0         0.253        0.194       0.812            0.657      0.815   
1         0.248        0.186       0.812            0.657      0.816   
2         0.258        0.195       0.812            0.657      0.816   
3         0.247        0.187       0.812            0.657      